In [1]:
import pandas as pd
import plotly.express as px
from plotly.io import to_image
import numpy as np

In [2]:
def load_df(file_name : str, num_users : int):
    path = file_name + '.csv'
    df = pd.read_csv(path)
    new_col_name = 'Average Response Time (ms) ' +  str(num_users)+'_usr'
    #df.set_index('Name', inplace=True, drop=False)
    df.drop(df.tail(1).index , inplace = True)
    df.rename(columns = {'Average Response Time':new_col_name}, inplace = True)
    df = df[[new_col_name]].apply(lambda x : np.log(x))
    df = df.iloc[:150]

    return df


In [3]:
el_usr1 = load_df('results/EL_usr1_spwr1_st0', 1)

el_usr2 = load_df('results/EL_usr2_spwr1_st0', 2)

merged_pd = pd.concat(
    [
        el_usr1, 
        el_usr2
    ], axis = 1)

el_usr5 = load_df('results/EL_usr5_spwr1_st0', 5)
merged_pd = pd.concat(
    [
        merged_pd,
        el_usr5
    ], axis = 1)

el_usr10 = load_df('results/EL_usr10_spwr1_st0', 10)
merged_pd = pd.concat(
    [
        merged_pd,
        el_usr10
    ], axis = 1)

el_usr20 = load_df('results/EL_usr20_spwr1_st0', 20)
merged_pd = pd.concat(
    [
        merged_pd,
        el_usr20
    ], axis = 1)

el_usr50 = load_df('results/EL_usr50_spwr1_st0', 50)
merged_pd = pd.concat(
    [
        merged_pd,
        el_usr50
    ], axis = 1)

In [4]:
label_dict = {
    "index" : "Execution Time"
}

color_sequence = ['red', 'green', 'gray', 'purple', 'orange', 'blue']

In [5]:
fig_lineplot = px.line(data_frame=merged_pd, labels = label_dict,  color_discrete_sequence=color_sequence)

In [6]:
fig_lineplot.show()

In [7]:
fig_boxplot = px.box(data_frame=merged_pd, labels = label_dict)

In [8]:
fig_boxplot.show()

In [14]:
with open(file='aggregated_lineplot.pdf', mode = 'wb') as f:
    f.write(to_image(fig_lineplot, format = 'pdf', scale = 1.0, width=1024))

In [16]:
with open(file='aggregated_boxplot.pdf', mode = 'wb') as f:
    f.write(to_image(fig_boxplot, format = 'pdf', scale = 1.0))